In [ ]:
import pandas as pd
import os
import flask

In [5]:
#I got this from google, but just ensuring the file is not corrupted and will open

df = pd.read_csv('Transactions.csv')

print(df.to_string())

         Date           Description  Amount      Type        Category
0    4/9/2025                 JACKS   29.00  Purchase  Food/Groceries
1    4/8/2025  WAL-MART SUPERCENTER    8.78  Purchase   Miscellaneous
2    4/5/2025          AMC THEATERS   35.78  Purchase   Entertainment
3    4/3/2025     PAYPAL *CHEWY INC   77.92  Purchase   Miscellaneous
4    4/1/2025             BRAVO #16   25.17  Purchase  Transportation
5    4/1/2025  WAL-MART SUPERCENTER   10.75  Purchase   Miscellaneous
6   3/28/2025  WAL-MART SUPERCENTER   27.07  Purchase   Miscellaneous
7   3/22/2025                SUNOCO   25.98  Purchase  Transportation
8   3/12/2025               TJ MAXX   61.76  Purchase   Miscellaneous
9   3/12/2025             PAPAJOHNS   16.49  Purchase  Food/Groceries
10  3/12/2025     PAYPAL *FOREVER21   73.22  Purchase   Miscellaneous
11  3/12/2025               SEPHORA   88.41  Purchase   Miscellaneous
12  3/12/2025                  ALDI  101.76  Purchase  Food/Groceries
13   3/9/2025       

In [8]:
CSV_FILE = 'Transactions.csv'
INITIAL_BALANCE = 100.0

class AccountManager:
    def __init__(self,initial_balance = INITIAL_BALANCE,csv_file = CSV_FILE):
            self.csv_file = csv_file
            self.initial_balance = initial_balance


    def load_file(self):
            """Loads data from the CSV file"""
            if os.path.exists(self.csv_file):
                try:
                    df = pd.read(self.csv_file)
                    print(f"File exists as {self.csv_file}")
                    return df
                except pd.errors.EmptyDataError:        #I did look this except up on google but it seems to do what I'm wanting
                    print("Error: The file is empty.")
            else:
                print(f"{self.csv_file} File not found")



AccountManager('Transactions.csv')